# **Python Implementation of binary classification of Brain Tumours and evaluating the model on different activation functions and Test train splits.**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Dictionary of optimizers used for training**

In [ ]:
optimizer_dict={
  1: 'SGD',  
  2: 'RMSprop',
  3: 'Adam', 
  4: 'Adadelta',
  5: 'Adagrad',
  6: 'Adamax',    
  7: 'Nadam'  
}

# **Image Pre-processing** 

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import cv2
import os
import matplotlib.image as mpimg
from zipfile import ZipFile

import warnings
warnings.filterwarnings('ignore')

from tensorflow import keras
from keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.utils import load_img,img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory



data_path = os.listdir(r"/kaggle/input/braintumour/BT/Testing")

path = r'/kaggle/input/braintumour/BT/Training'
classes = os.listdir(path)
classes
fig = plt.gcf()
fig.set_size_inches(16, 16)

notum_dir = os.path.join(r"/kaggle/input/braintumour/BT/Training/notumor")
tum_dir = os.path.join(r"/kaggle/input/braintumour/BT/Training/tumor")
notum_names = os.listdir(notum_dir)
tum_names = os.listdir(tum_dir)

pic_index = 210

notum_images = [os.path.join(notum_dir, fname)
            for fname in notum_names[pic_index-8:pic_index]]
tum_images = [os.path.join(tum_dir, fname)
            for fname in tum_names[pic_index-8:pic_index]]

for i, img_path in enumerate(tum_images + notum_images):
    sp = plt.subplot(4, 4, i+1)
    sp.axis('Off')

base_dir = r'/kaggle/input/braintumour/BT/Training'


train_datagen = image_dataset_from_directory(base_dir,
                                                image_size=(200,200),
                                                subset='training',
                                                seed = 1,
                                                validation_split=0.3,
                                                batch_size= 32)
test_datagen = image_dataset_from_directory(base_dir,
                                                image_size=(200,200),
                                                subset='validation',
                                                seed = 1,
                                                validation_split=0.3,
                                                batch_size= 32)



# **Model Definition**

In [ ]:
model = tf.keras.models.Sequential([
    layers.Conv2D(256, (2, 2), activation='relu', input_shape=(200, 200, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (2, 2), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (2, 2), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (2, 2), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.1),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(1, activation='sigmoid')
])
model.summary()
'''optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)
history = model.fit(train_datagen,
        epochs=2,
        validation_data=test_datagen)
data_columns=["Optimizer", "Epoch", "Training Loss"," Training Accuracy","Validation Loss","Validation Accuracy"]
data_df=pd.DataFrame(columns=data_columns)
history_df = pd.DataFrame(history.history)
data_df = pd.DataFrame(history_df.values, columns=data_df.columns[2:])
data_df.to_csv('mycsvfile.csv', mode='a', header=False, index=False)

history_df.loc[:, ['loss', 'val_loss']].plot()
plt.xlim(0, 10)
plt.ylim(0, 1)
plt.show()

'''

In [ ]:
dot_img_file = 'model_1.png'
tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

# **Performance Metrics Evaluation**

In [ ]:
with open('modelsummary.txt', 'w') as f:

    model.summary(print_fn=lambda x: f.write(x + '\n'))

In [ ]:
def calculate_recall_precision_sensitivity_specificity(confusion_matrix):
    true_positives = confusion_matrix[1][1]
    false_positives = confusion_matrix[0][1]
    false_negatives = confusion_matrix[1][0]
    true_negatives = confusion_matrix[0][0]

    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
    sensitivity = recall
    specificity = true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) != 0 else 0

    return recall, precision, sensitivity, specificity



# **Model Training and Results**

In [ ]:
import csv
import pandas as pd
from keras import backend
from sklearn.metrics import confusion_matrix
csv_columns = ["Optimizer", "Epoch", "Training Loss", "Training Accuracy", "Validation Loss", "Validation Accuracy", "Sensitivity", "Specificity", "Precision", "Recall"]
csv_df = pd.DataFrame(columns=csv_columns)
csv_df.to_csv('Sigmoid 70-30.csv', index=False)

model.compile(
loss='binary_crossentropy',
optimizer=optimizer_dict[3],
metrics=['accuracy']
        )
model.save_weights('model.h5')
print("Currently Training for ",20,"Epoch(s).")
history = model.fit(train_datagen,
epochs=20,
validation_data=test_datagen)
history_df = pd.DataFrame(history.history)
test_generator = test_datagen.unbatch().batch(900000000)
test_images, test_labels = next(iter(test_generator))
predictions = model.predict(test_images)
predicted_classes = np.round(predictions)
cm = confusion_matrix(test_labels, predicted_classes)
recall, precision, sensitivity, specificity = calculate_recall_precision_sensitivity_specificity(cm)
save_path="model"+str(optimizer_dict[i])+".h5"
model.save_weights(save_path)
model.load_weights('model.h5') 
print("ALL DONE")

In [ ]:
def swap_diagonal(matrix):
    if len(matrix) != 2 or len(matrix[0]) != 2 or len(matrix[1]) != 2:
        print("Error: Input matrix must be a 2x2 matrix.")
        return None

    temp = matrix[0][0]
    matrix[0][0] = matrix[1][1]
    matrix[1][1] = temp

    return matrix

In [ ]:
cm=swap_diagonal(cm)

In [ ]:
labels = ['Tumour', 'NoTumour']

# Create a seaborn heatmap
sns.set(font_scale=1.4)  # Adjust font scale if necessary
plt.figure(figsize=(6, 4))  # Adjust figure size if necessary
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', annot_kws={"size": 16},
            xticklabels=labels, yticklabels=labels)

# Add labels and title
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')

# Show plot
plt.show()

In [ ]:
my_model = tf.keras.models.Sequential([
    layers.Conv2D(256, (2, 2), activation='relu', input_shape=(200, 200, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (2, 2), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (2, 2), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (2, 2), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.1),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(1, activation='sigmoid')
])

for i in optimizer_dict:
    path='model'+optimizer_dict[i]+'.h5'
    my_model.load_weights(path)
    max_weight = float('-inf')
    min_weight = float('inf')

    # Iterate through each layer of the model
    for layer in model.layers:
        # Get the weights of the layer
        weights = layer.get_weights()
        if len(weights) > 0:
            # Iterate through each weight matrix in the layer
            for weight_matrix in weights:
                # Update the maximum and minimum weights
                max_weight_in_matrix = tf.reduce_max(weight_matrix)
                min_weight_in_matrix = tf.reduce_min(weight_matrix)
                if max_weight_in_matrix > max_weight:
                    max_weight = max_weight_in_matrix
                if min_weight_in_matrix < min_weight:
                    min_weight = min_weight_in_matrix

    print("Range for 70-30 split for the ",optimizer_dict[i], max_weight,"-", min_weight)


# **Blob Detection Using Open CV**

In [ ]:

imgExtension = ["png", "jpeg", "jpg"] #Image Extensions to be chosen from
allImages = list()



def chooseRandomImage(directory=r"/kaggle/input/braintumour/BT/Testing"):
    for img in os.listdir(directory): #Lists all files
        ext = img.split(".")[len(img.split(".")) - 1]
        if (ext in imgExtension):
            allImages.append(img)
    choice = random.randint(0, len(allImages) - 1)
    chosenImage = allImages[choice] #Do Whatever you want with the image file
    return chosenImage

randomImage = chooseRandomImage()


#Input image
test_image = load_img('/kaggle/input/braintumour/BT/Testing//'+randomImage,target_size=(200,200))
path='/kaggle/input/braintumour/BT/Testing//'+randomImage

test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)


result = model.predict(test_image)

#Mapping result array with the main name list
i=0
if(result>=0.5):
    print(" Tumor found")
else:
    print("No tumor found")
img = cv2.imread(path)  

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to reduce noise  
blur = cv2.GaussianBlur(gray, (5,5), 0)

# Perform blob detection
params = cv2.SimpleBlobDetector_Params()
params.filterByColor = True
params.blobColor = 255 # Detect brighter white blobs

# Filter by circularity to detect rounder blobs
params.filterByCircularity = True 
params.minCircularity = 0.2

# Filter by convexity to detect convex blobs
params.filterByConvexity = True
params.minConvexity = 0.2

# Create blob detector with custom params
detector = cv2.SimpleBlobDetector_create(params)

# Detect brighter blobs 
keypoints = detector.detect(blur)
params.filterByArea = True
params.minArea = 500
params.maxArea = 2000



detector = cv2.SimpleBlobDetector_create(params)
keypoints = detector.detect(blur)

# Draw boxes around detected blobs
for k in keypoints:
    x, y, r = int(k.pt[0]), int(k.pt[1]), int(k.size/2)
    cv2.rectangle(img, (x-r, y-r), (x+r, y+r), (0,0,255), 2)

# Show result
plt.imshow(img) 



